In [ ]:
# Create a working feature dataset
df_features = df_raw.copy()

print("Feature dataset rows:", len(df_features))
df_features.head()


In [ ]:
# =====================================================
# STAGE 2: CALCULATE ALL HEALTHCARE STRESS INDICES
# =====================================================

# IMPORTANT:
# - df_raw is NEVER modified
# - df_features is the working dataframe

df_features = df_raw.copy()

# -------------------------------
# CONFIGURABLE CLINICAL ASSUMPTIONS
# -------------------------------
VENTILATION_RATE = 0.05   # 5% of IP patients need ventilators (realistic)
ICU_RATE = 0.15           # 15% of IP patients need ICU (proxy)

# -------------------------------
# CAPACITY-BASED STRESS INDICES
# -------------------------------

# 1️⃣ Bed Stress Index
df_features["bed_stress_index"] = (
    df_features["ip_demand"] / df_features["total_beds"]
)

# 2️⃣ ICU Stress Index (proxy-based)
df_features["icu_stress_index"] = (
    (df_features["ip_demand"] * ICU_RATE) / df_features["icu_beds"]
)

# 3️⃣ Ventilator Stress Index (FIXED & REALISTIC)
df_features["vent_stress_index"] = (
    (df_features["ip_demand"] * VENTILATION_RATE) /
    (
        df_features["ventilators_invasive"] +
        df_features["ventilators_non_invasive"]
    )
)

# -------------------------------
# STAFF & SERVICE LOAD STRESS
# -------------------------------

# 4️⃣ Doctor Load Stress (IP per institution)
df_features["doctor_load"] = (
    df_features["doctor_load_proxy_ip_per_institution"]
)

# 5️⃣ Nurse Load Stress (IP per bed)
df_features["nurse_load"] = (
    df_features["nurse_load_proxy_ip_per_bed"]
)

# 6️⃣ OP Load Stress (OP pressure on beds)
df_features["op_load"] = (
    df_features["op_load_proxy_op_per_bed"]
)

# -------------------------------
# EMERGENCY ACCESS STRESS
# -------------------------------

# 7️⃣ Ambulance Risk Index (inverse availability proxy)
df_features["ambulance_risk"] = (
    df_features["ip_demand"] / df_features["total_ambulance"]
)

# -------------------------------
# FINAL CHECK
# -------------------------------
df_features[
    [
        "district",
        "bed_stress_index",
        "icu_stress_index",
        "vent_stress_index",
        "doctor_load",
        "nurse_load",
        "op_load",
        "ambulance_risk"
    ]
].head()


In [ ]:
# =====================================================
# STAGE 2 – BED STRESS INDEX BY DISTRICT
# =====================================================

import matplotlib.pyplot as plt

plot_df = df_features.sort_values(
    "bed_stress_index", ascending=False
)

plt.figure(figsize=(12,5))

plt.bar(
    plot_df["district"],
    plot_df["bed_stress_index"]
)

plt.axhline(
    y=1.0,
    color="red",
    linestyle="--",
    label="Capacity Limit (Stress = 1.0)"
)

plt.xlabel("District")
plt.ylabel("Bed Stress Index (IP Demand / Beds)")
plt.title("Stage 2: Bed Stress Index by District")
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()



In [ ]:
# =====================================================
# VENTILATOR STRESS INDEX BY DISTRICT
# =====================================================

plot_df = df_features.sort_values(
    "vent_stress_index", ascending=False
)

plt.figure(figsize=(12,5))

plt.bar(
    plot_df["district"],
    plot_df["vent_stress_index"],
    color="green"
)

plt.axhline(
    y=1.0,
    color="red",
    linestyle="--",
    label="Capacity Limit"
)

plt.xlabel("District")
plt.ylabel("Ventilator Stress Index")
plt.title("Stage 2: Ventilator Stress Index by District")
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# DOCTOR LOAD STRESS BY DISTRICT
# =====================================================

plot_df = df_features.sort_values(
    "doctor_load", ascending=False
)

plt.figure(figsize=(12,5))

plt.bar(
    plot_df["district"],
    plot_df["doctor_load"],
    color="purple"
)

plt.xlabel("District")
plt.ylabel("Doctor Load (IP per Institution)")
plt.title("Stage 2: Doctor Load Stress by District")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# NURSE LOAD STRESS BY DISTRICT
# =====================================================

plot_df = df_features.sort_values(
    "nurse_load", ascending=False
)

plt.figure(figsize=(12,5))

plt.bar(
    plot_df["district"],
    plot_df["nurse_load"],
    color="teal"
)

plt.xlabel("District")
plt.ylabel("Nurse Load (IP per Bed)")
plt.title("Stage 2: Nurse Load Stress by District")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# AMBULANCE RISK BY DISTRICT
# =====================================================

plot_df = df_features.sort_values(
    "ambulance_risk", ascending=False
)

plt.figure(figsize=(12,5))

plt.bar(
    plot_df["district"],
    plot_df["ambulance_risk"],
    color="brown"
)

plt.xlabel("District")
plt.ylabel("Ambulance Risk Index")
plt.title("Stage 2: Emergency Access Risk by District")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# STAGE 2: STRESS HEATMAP (ALL INDICES)
# =====================================================

import seaborn as sns

stress_cols = [
    "bed_stress_index",
    "icu_stress_index",
    "vent_stress_index",
    "doctor_load",
    "nurse_load",
    "ambulance_risk"
]

plt.figure(figsize=(10,6))

sns.heatmap(
    df_features.set_index("district")[stress_cols],
    cmap="Reds",
    linewidths=0.5
)

plt.title("Stage 2: Healthcare Stress Heatmap by District")
plt.ylabel("District")
plt.xlabel("Stress Indicators")
plt.tight_layout()
plt.show()


# Individual stress indices

In [ ]:

# -------------------------------
# INDIVIDUAL STRESS INDICES
# -------------------------------

# Bed Stress Index
df_features["bed_stress_index"] = (
    df_features["ip_demand"] / df_features["total_beds"]
)

# ICU Stress Index
df_features["icu_stress_index"] = (
    df_features["ip_demand"] / df_features["icu_beds"]
)

# Ventilator Stress Index
df_features["vent_stress_index"] = (
    df_features["ip_demand"] / (
        df_features["ventilators_invasive"] +
        df_features["ventilators_non_invasive"]
    )
)

# -------------------------------
# COMPOSITE DEMAND–RISK INDEX
# -------------------------------
df_features["demand_risk_index"] = (
    0.5 * df_features["bed_stress_index"] +
    0.3 * df_features["icu_stress_index"] +
    0.2 * df_features["vent_stress_index"]
)

df_features[
    ["district", "bed_stress_index", "icu_stress_index",
     "vent_stress_index", "demand_risk_index"]
].head()


#Normalize all indices (0–1 scale)

In [ ]:
df_features_scaled = df_features.copy()


In [ ]:
from sklearn.preprocessing import StandardScaler

# Columns to scale
scale_cols = [
    "bed_stress_index",
    "icu_stress_index",
    "vent_stress_index",
    "demand_risk_index"
]

scaler = StandardScaler()

df_features_scaled[scale_cols] = scaler.fit_transform(
    df_features_scaled[scale_cols]
)

df_features_scaled[
    ["district"] + scale_cols
].head()


#Computing the Demand–Risk Index (core)
#used weighted aggregation

In [ ]:
# =====================================================
# BLOCK 1: CREATE INDIVIDUAL RISK / LOAD INDICATORS
# =====================================================

# Doctor workload risk (higher = more stressed)
df_features["doctor_load"] = (
    df_features["doctor_load_proxy_ip_per_institution"] /
    df_features["doctor_load_proxy_ip_per_institution"].max()
)

# Nurse workload risk
df_features["nurse_load"] = (
    df_features["nurse_load_proxy_ip_per_bed"] /
    df_features["nurse_load_proxy_ip_per_bed"].max()
)

# Ventilator risk (critical care pressure)
df_features["ventilator_risk"] = (
    df_features["ip_demand"] /
    (
        df_features["ventilators_invasive"] +
        df_features["ventilators_non_invasive"]
    )
)

# Ambulance risk (emergency access pressure)
df_features["ambulance_risk"] = (
    df_features["ip_demand"] /
    df_features["total_ambulance"]
)

df_features[
    ["district", "doctor_load", "nurse_load",
     "ventilator_risk", "ambulance_risk"]
].head()


In [ ]:
# =====================================================
# BLOCK 2: NORMALIZE + WEIGHTED AGGREGATION
# =====================================================

# Risk components to normalize
risk_components = [
    "bed_stress_index",
    "icu_stress_index",
    "doctor_load",
    "nurse_load",
    "ventilator_risk",
    "ambulance_risk"
]

# Min–Max Normalization
df_features[risk_components] = (
    df_features[risk_components] - df_features[risk_components].min()
) / (
    df_features[risk_components].max() - df_features[risk_components].min()
)

# Weighted aggregation
df_features["demand_risk_index"] = (
    0.25 * df_features["bed_stress_index"] +
    0.25 * df_features["icu_stress_index"] +
    0.15 * df_features["doctor_load"] +
    0.15 * df_features["nurse_load"] +
    0.10 * df_features["ventilator_risk"] +
    0.10 * df_features["ambulance_risk"]
)

df_features[
    ["district", "demand_risk_index"]
].sort_values("demand_risk_index", ascending=False)


#Risk categorization

In [ ]:
# =====================================================
# RISK CATEGORIZATION + DISTRICT RANKING
# =====================================================

# Risk category function
def risk_category(score):
    if score >= 0.7:
        return "High Risk"
    elif score >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

# Apply to FEATURE dataset (NOT df_raw)
df_features["risk_category"] = df_features["demand_risk_index"].apply(risk_category)

# Rank districts by risk
risk_ranking = df_features.sort_values(
    "demand_risk_index", ascending=False
)[["district", "demand_risk_index", "risk_category"]]

risk_ranking


In [ ]:
# =====================================================
# BLOCK 1: RAW STRESS VALUES FOR EACH SCENARIO
# =====================================================

df_scenario = df_features.copy()

# Scenario demands
df_scenario["ip_normal"] = df_scenario["ip_demand"]
df_scenario["ip_peak"]   = df_scenario["ip_demand"] * 1.25
df_scenario["ip_surge"]  = df_scenario["ip_demand"] * 1.60

# Raw stress calculations
def raw_stress(ip, beds, icu):
    return 0.5 * (ip / beds) + 0.5 * (ip / icu)

df_scenario["stress_normal"] = raw_stress(
    df_scenario["ip_normal"],
    df_scenario["total_beds"],
    df_scenario["icu_beds"]
)

df_scenario["stress_peak"] = raw_stress(
    df_scenario["ip_peak"],
    df_scenario["total_beds"],
    df_scenario["icu_beds"]
)

df_scenario["stress_surge"] = raw_stress(
    df_scenario["ip_surge"],
    df_scenario["total_beds"],
    df_scenario["icu_beds"]
)

df_scenario[
    ["district", "stress_normal", "stress_peak", "stress_surge"]
].head()


In [ ]:
# =====================================================
# BLOCK 2: DISTRICT-WISE TIPPING POINT
# =====================================================

def tipping_point(row, threshold=1.0):
    if row["stress_normal"] >= threshold:
        return "Normal"
    elif row["stress_peak"] >= threshold:
        return "Peak"
    elif row["stress_surge"] >= threshold:
        return "Surge"
    else:
        return "No Failure"

df_scenario["tipping_point"] = df_scenario.apply(tipping_point, axis=1)

tipping_table = df_scenario[
    ["district", "stress_normal", "stress_peak", "stress_surge", "tipping_point"]
].sort_values("tipping_point")

tipping_table


In [ ]:
# =====================================================
# BLOCK 1: RISK CATEGORY CREATION (SAFETY CHECK)
# =====================================================

def risk_category(score):
    if score >= 0.7:
        return "High Risk"
    elif score >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

df_scenario["risk_category"] = df_scenario["demand_risk_index"].apply(risk_category)

df_scenario[
    ["district", "demand_risk_index", "risk_category"]
].head()


In [ ]:
# =====================================================
# BLOCK 2: RISK CATEGORY vs TIPPING POINT TABLE
# =====================================================

risk_tipping_table = df_scenario[
    ["district", "risk_category", "tipping_point"]
].sort_values(
    ["risk_category", "tipping_point"]
)

risk_tipping_table


In [ ]:
# =====================================================
# BLOCK 3: CROSSTAB – RISK vs FAILURE STAGE
# =====================================================

risk_vs_failure = pd.crosstab(
    df_scenario["risk_category"],
    df_scenario["tipping_point"]
)

risk_vs_failure


In [ ]:
# =====================================================
# BLOCK 4: BAR PLOT – RISK CATEGORY vs FAILURE
# =====================================================

risk_vs_failure.plot(
    kind="bar",
    figsize=(10,6)
)

plt.xlabel("Risk Category")
plt.ylabel("Number of Districts")
plt.title("Failure Stage Distribution by Risk Category")
plt.xticks(rotation=0)
plt.legend(title="Failure Stage")
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# BLOCK 1: FILTER HIGH-RISK DISTRICTS (SURGE SCENARIO)
# =====================================================

high_risk_df = df_scenario[
    df_scenario["risk_category"] == "High Risk"
].copy()

high_risk_df[
    ["district", "risk_category", "stress_surge"]
]


In [ ]:
# =====================================================
# ADD STATE-LEVEL EMERGENCY BED POOL
# =====================================================

emergency_pool = 0.10 * high_risk_df["total_beds"].sum()  # 10% surge beds


In [ ]:
# =====================================================
# BLOCK 1: RISK-AWARE SURGE OPTIMIZATION (LP)
# =====================================================

import pulp
import numpy as np

# Scenario dataframe (safe copy)
df_scenario = df_features.copy()

# Surge demand
df_scenario["ip_surge"] = df_scenario["ip_demand"] * 1.60

# Inputs
districts = df_scenario["district"].tolist()

ip_surge = dict(zip(df_scenario["district"], df_scenario["ip_surge"]))
risk = dict(zip(df_scenario["district"], df_scenario["demand_risk_index"]))
current_beds = dict(zip(df_scenario["district"], df_scenario["total_beds"]))

TOTAL_STATE_BEDS = sum(current_beds.values())

# Baseline unmet demand (before optimization)
unmet_before = {
    d: max(ip_surge[d] - current_beds[d], 0)
    for d in districts
}

# LP model
model = pulp.LpProblem(
    "Risk_Aware_Surge_Bed_Optimization",
    pulp.LpMaximize
)

# Decision variables
beds_alloc = {
    d: pulp.LpVariable(f"beds_alloc_{d}", lowBound=0)
    for d in districts
}

unmet_after = {
    d: pulp.LpVariable(
        f"unmet_after_{d}",
        lowBound=0,
        upBound=unmet_before[d]  # cannot worsen
    )
    for d in districts
}

# Objective: risk-weighted reduction in unmet demand
model += pulp.lpSum(
    risk[d] * (unmet_before[d] - unmet_after[d])
    for d in districts
)

# Constraints
for d in districts:
    model += unmet_after[d] >= ip_surge[d] - beds_alloc[d]

# State-level bed pool constraint
model += pulp.lpSum(beds_alloc[d] for d in districts) <= TOTAL_STATE_BEDS

# Solve
model.solve(pulp.PULP_CBC_CMD(msg=False))

# Store results
df_scenario["beds_allocated_surge_optimal"] = df_scenario["district"].apply(
    lambda d: beds_alloc[d].varValue
)

df_scenario["unmet_surge_before"] = df_scenario["district"].apply(
    lambda d: unmet_before[d]
)

df_scenario["unmet_surge_after"] = df_scenario["district"].apply(
    lambda d: unmet_after[d].varValue
)

df_scenario[
    [
        "district",
        "ip_surge",
        "total_beds",
        "beds_allocated_surge_optimal",
        "unmet_surge_before",
        "unmet_surge_after"
    ]
]


In [ ]:
# =====================================================
# BLOCK 2: VISUALIZATION (BEFORE vs AFTER)
# =====================================================

import matplotlib.pyplot as plt

# Sort for clarity
plot_df = df_scenario.sort_values(
    "unmet_surge_before", ascending=False
)

districts = plot_df["district"]
before = plot_df["unmet_surge_before"]
after = plot_df["unmet_surge_after"]

x = np.arange(len(districts))
width = 0.35

plt.figure(figsize=(15,6))

plt.bar(
    x - width/2,
    before,
    width,
    label="Before Optimization (Surge)",
    color="steelblue"
)

plt.bar(
    x + width/2,
    after,
    width,
    label="After Optimization (Surge)",
    color="darkorange"
)

plt.xticks(x, districts, rotation=45)
plt.ylabel("Unmet Inpatient Demand")
plt.xlabel("District")
plt.title("Surge Scenario: Unmet Inpatient Demand Before vs After Optimization")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# KERALA HEALTHCARE STRESS TESTING – EXTREME SURGES
# =====================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------
# WORKING DATAFRAME
# -------------------------------
df_stress = df_features.copy()

# -------------------------------
# CONFIGURATION
# -------------------------------
SURGE_SCENARIOS = {
    "Baseline (1.0x)": 1.0,
    "Mild Surge (1.2x)": 1.2,
    "Moderate Surge (1.4x)": 1.4,
    "Severe Surge (1.6x)": 1.6,
    "Extreme Surge (1.8x)": 1.8,
    "Collapse Zone (2.0x)": 2.0
}

RISK_COL = "demand_risk_index"

# -------------------------------
# BASE DEMAND & CAPACITY
# -------------------------------
df_stress["bed_capacity"] = df_stress["total_beds"]
df_stress["icu_capacity"] = df_stress["icu_beds"]
df_stress["vent_capacity"] = (
    df_stress["ventilators_invasive"] +
    df_stress["ventilators_non_invasive"]
)

# Demand proxies (standard in healthcare planning)
df_stress["bed_demand_base"] = df_stress["ip_demand"]
df_stress["icu_demand_base"] = df_stress["ip_demand"] * 0.15
df_stress["vent_demand_base"] = df_stress["ip_demand"] * 0.05

# -------------------------------
# STRESS TEST LOOP
# -------------------------------
results = []

for scenario, factor in SURGE_SCENARIOS.items():

    bed_demand = df_stress["bed_demand_base"] * factor
    icu_demand = df_stress["icu_demand_base"] * factor
    vent_demand = df_stress["vent_demand_base"] * factor

    bed_unmet = np.maximum(bed_demand - df_stress["bed_capacity"], 0)
    icu_unmet = np.maximum(icu_demand - df_stress["icu_capacity"], 0)
    vent_unmet = np.maximum(vent_demand - df_stress["vent_capacity"], 0)

    results.append({
        "Scenario": scenario,
        "Surge_Factor": factor,
        "Total_Bed_Unmet": bed_unmet.sum(),
        "Total_ICU_Unmet": icu_unmet.sum(),
        "Total_Vent_Unmet": vent_unmet.sum(),
        "High_Risk_Districts": (bed_unmet > 0).sum()
    })

stress_df = pd.DataFrame(results)

# -------------------------------
# DISPLAY SUMMARY TABLE
# -------------------------------
stress_df


In [ ]:
# =====================================================
# PLOT: UNMET DEMAND vs SURGE LEVEL
# =====================================================

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

plt.plot(
    stress_df["Surge_Factor"],
    stress_df["Total_Bed_Unmet"],
    marker="o",
    linewidth=2,
    label="Beds"
)

plt.plot(
    stress_df["Surge_Factor"],
    stress_df["Total_ICU_Unmet"],
    marker="s",
    linewidth=2,
    label="ICU"
)

plt.plot(
    stress_df["Surge_Factor"],
    stress_df["Total_Vent_Unmet"],
    marker="^",
    linewidth=2,
    label="Ventilators"
)

# Labels & title
plt.xlabel("Surge Multiplier (Demand Increase)")
plt.ylabel("Total Unmet Demand")
plt.title("Kerala Healthcare Stress Test: Capacity Failure Curve")

# Grid & legend
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()

plt.tight_layout()
plt.show()


